# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [ ]:
import pandas as pd
import requests
import math
import time
from scipy import stats
from statistics import mean

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [179]:
stocks = pd.read_csv('sp_500_stocks.csv')

from secretsRENAMED import FINNHUB_API_KEY

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [180]:
symbol = 'AAPL'

basic_financials_api_url = f'https://finnhub.io/api/v1/stock/metric?symbol={symbol}&metric=all&token={FINNHUB_API_KEY}'
pricing_data = requests.get(basic_financials_api_url).json()

pricing_data

{'metric': {'10DayAverageTradingVolume': 49.13645,
  '13WeekPriceReturnDaily': 8.5213,
  '26WeekPriceReturnDaily': 35.8859,
  '3MonthADReturnStd': 17.694698,
  '3MonthAverageTradingVolume': 49.10214,
  '52WeekHigh': 288.62,
  '52WeekHighDate': '2025-12-03',
  '52WeekLow': 169.2101,
  '52WeekLowDate': '2025-04-08',
  '52WeekPriceReturnDaily': 7.2629,
  '5DayPriceReturnDaily': 0.5952,
  'assetTurnoverAnnual': 1.1584,
  'assetTurnoverTTM': 1.2186,
  'beta': 1.0966198,
  'bookValuePerShareAnnual': 4.991,
  'bookValuePerShareQuarterly': 4.991,
  'bookValueShareGrowth5Y': 5.34,
  'capexCagr5Y': 11.71,
  'cashFlowPerShareAnnual': 6.6855,
  'cashFlowPerShareQuarterly': 6.6855,
  'cashFlowPerShareTTM': 6.86253,
  'cashPerSharePerShareAnnual': 3.7024,
  'cashPerSharePerShareQuarterly': 3.7024,
  'currentDividendYieldTTM': 0.3812,
  'currentEv/freeCashFlowAnnual': 41.5993,
  'currentEv/freeCashFlowTTM': 41.5993,
  'currentRatioAnnual': 0.8933,
  'currentRatioQuarterly': 0.8933,
  'dividendGrowthR

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [181]:
pricing_data['metric']['52WeekPriceReturnDaily']

7.2629

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [182]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_url = f'https://finnhub.io/api/v1/quote?symbol={symbol_string}&token={FINNHUB_API_KEY}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['c'],
                    'N/A',
                    'N/A'
                ],
                index = my_columns
            ),
            ignore_index = True
        )

final_dataframe

######## Finnhub does not support batch API requests ########

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

In [35]:
final_dataframe = pd.DataFrame(columns = my_columns)
rows = []
for stock in stocks['Ticker']:
    try:
        pricing_api_url = f'https://finnhub.io/api/v1/quote?symbol={stock}&token={FINNHUB_API_KEY}'
        basic_financials_api_url = f'https://finnhub.io/api/v1/stock/metric?symbol={stock}&metric=all&token={FINNHUB_API_KEY}'

        pricing_data = requests.get(pricing_api_url).json()
        if pricing_data.get('error'):
            raise Exception(f"Error fetching pricing data for {stock}: {pricing_data['error']}")

        basic_financials = requests.get(basic_financials_api_url).json()
        if basic_financials.get('error'):
            raise Exception(f"Error fetching basic financials data for {stock}: {basic_financials['error']}")

        price = pricing_data.get('c') or 'N/A'
        one_year_price_return = basic_financials.get('metric', {}).get('52WeekPriceReturnDaily') or 'N/A'

        price = price if price != 0 else 'N/A'
        one_year_price_return = one_year_price_return if one_year_price_return != 0 else 'N/A'

    except Exception as e:
        print(f"An error occurred for {stock}: {e}")
        price = 'N/A'
        one_year_price_return = 'N/A'
    
    rows.append([stock, price, one_year_price_return, 'N/A'])
    
    time.sleep(1)

final_dataframe = pd.DataFrame(rows, columns=my_columns)

final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,138.32,2.7637,N/A
1,AAL,15.68,-9.1014,N/A
2,AAP,41.12,-6.0114,N/A
3,AAPL,273.81,7.2629,N/A
4,ABBV,229.89,28.8693,N/A
...,...,...,...,...
500,YUM,154.32,15.2846,N/A
501,ZBH,90.23,-15.5308,N/A
502,ZBRA,245.9,-37.0536,N/A
503,ZION,59.55,9.5072,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [183]:
final_dataframe_updated = final_dataframe[final_dataframe['One-Year Price Return'] != 'N/A']

final_dataframe_updated = final_dataframe_updated.sort_values('One-Year Price Return', ascending=False)
final_dataframe_updated = final_dataframe_updated[:50]
final_dataframe_updated.reset_index(inplace=True, drop=True)
final_dataframe_updated

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,WDC,179.56,286.0161,N/A
1,STX,285.27,222.2298,N/A
2,MU,286.68,219.5274,N/A
3,NEM,104.73,174.4497,N/A
4,LRCX,177.33,140.2846,N/A
5,TPR,130.2,101.1432,N/A
6,KLAC,1276.99,97.0694,N/A
7,APH,137.94,94.3091,N/A
8,HWM,212.17,91.9396,N/A
9,GLW,89.66,88.7579,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [184]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the value of your portfolio:')
    
    try:
        float(portfolio_size)
    except ValueError:
        print("That's not a number! Please try again")

portfolio_input()
print(portfolio_size)

1000000


In [185]:
position_size = float(portfolio_size) / len(final_dataframe_updated.index)
for i in range(0, len(final_dataframe_updated.index)):
    final_dataframe_updated.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe_updated.loc[i, 'Price'])

final_dataframe_updated

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,WDC,179.56,286.0161,111
1,STX,285.27,222.2298,70
2,MU,286.68,219.5274,69
3,NEM,104.73,174.4497,190
4,LRCX,177.33,140.2846,112
5,TPR,130.2,101.1432,153
6,KLAC,1276.99,97.0694,15
7,APH,137.94,94.3091,144
8,HWM,212.17,91.9396,94
9,GLW,89.66,88.7579,223


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [198]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)
rows = []

for stock in stocks['Ticker']:
    try:
        pricing_api_url = f'https://finnhub.io/api/v1/quote?symbol={stock}&token={FINNHUB_API_KEY}'
        basic_financials_api_url = f'https://finnhub.io/api/v1/stock/metric?symbol={stock}&metric=all&token={FINNHUB_API_KEY}'

        pricing_data = requests.get(pricing_api_url).json()
        if pricing_data.get('error'):
            raise Exception(f"Error fetching pricing data for {stock}: {pricing_data['error']}")

        basic_financials = requests.get(basic_financials_api_url).json()
        if basic_financials.get('error'):
            raise Exception(f"Error fetching basic financials data for {stock}: {basic_financials['error']}")

        price = pricing_data.get('c') or 'N/A'
        one_year_price_return = basic_financials.get('metric', {}).get('52WeekPriceReturnDaily') or 'N/A'
        six_month_price_return = basic_financials.get('metric', {}).get('26WeekPriceReturnDaily') or 'N/A'
        three_month_price_return = basic_financials.get('metric', {}).get('13WeekPriceReturnDaily') or 'N/A'
        one_month_price_return = basic_financials.get('metric', {}).get('monthToDatePriceReturnDaily') or 'N/A'

        price = price if price != 0 else 'N/A'
        one_year_price_return = one_year_price_return/100 if one_year_price_return != 0 and one_year_price_return != 'N/A' else 'N/A'
        six_month_price_return = six_month_price_return/100 if six_month_price_return != 0 and six_month_price_return != 'N/A' else 'N/A'
        three_month_price_return = three_month_price_return/100 if three_month_price_return != 0 and three_month_price_return != 'N/A' else 'N/A'
        one_month_price_return = one_month_price_return/100 if one_month_price_return != 0 and one_month_price_return != 'N/A' else 'N/A'
    except Exception as e:
        print(f"An error occurred for {stock}: {e}")
        price = 'N/A'
        one_year_price_return = 'N/A'
        six_month_price_return = 'N/A'
        three_month_price_return = 'N/A'
        one_month_price_return = 'N/A'
    
    rows.append([stock, price, 'N/A', one_year_price_return, 'N/A', six_month_price_return, 'N/A', three_month_price_return, 'N/A', one_month_price_return, 'N/A', 'N/A'])
    
    time.sleep(1)

hqm_dataframe = pd.DataFrame(rows, columns=hqm_columns)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,138.32,N/A,0.027637,N/A,0.191079,N/A,0.112076,N/A,-0.098893,N/A,N/A
1,AAL,15.68,N/A,-0.091014,N/A,0.438532,N/A,0.313233,N/A,0.116014,N/A,N/A
2,AAP,41.12,N/A,-0.060114,N/A,-0.208165,N/A,-0.326454,N/A,-0.207402,N/A,N/A
3,AAPL,273.81,N/A,0.072629,N/A,0.358859,N/A,0.085213,N/A,-0.018074,N/A,N/A
4,ABBV,229.89,N/A,0.288693,N/A,0.251034,N/A,0.045002,N/A,0.009618,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,154.32,N/A,0.152846,N/A,0.091449,N/A,0.032586,N/A,0.007245,N/A,N/A
501,ZBH,90.23,N/A,-0.155308,N/A,-0.01388,N/A,-0.088402,N/A,-0.074754,N/A,N/A
502,ZBRA,245.9,N/A,-0.370536,N/A,-0.169958,N/A,-0.202866,N/A,-0.027102,N/A,N/A
503,ZION,59.55,N/A,0.095072,N/A,0.20376,N/A,0.046389,N/A,0.11873,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [199]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

hqm_dataframe_updated = hqm_dataframe[
    (hqm_dataframe['Price'] != 'N/A') &
    (hqm_dataframe['One-Year Price Return'] != 'N/A') &
    (hqm_dataframe['Six-Month Price Return'] != 'N/A') &
    (hqm_dataframe['Three-Month Price Return'] != 'N/A') &
    (hqm_dataframe['One-Month Price Return'] != 'N/A')
]

hqm_dataframe_updated = hqm_dataframe_updated.reset_index(drop=True)


for row in hqm_dataframe_updated.index:
    for time_period in time_periods:
        return_column = f'{time_period} Price Return'
        percentile_column = f'{time_period} Return Percentile'
        
        hqm_dataframe_updated.loc[row, percentile_column] = stats.percentileofscore(
            hqm_dataframe_updated[return_column],
            hqm_dataframe_updated.loc[row, return_column]
        )/100

hqm_dataframe_updated

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,138.32,N/A,0.027637,0.422907,0.191079,0.740088,0.112076,0.735683,-0.098893,0.03304,N/A
1,AAL,15.68,N/A,-0.091014,0.264317,0.438532,0.920705,0.313233,0.96696,0.116014,0.920705,N/A
2,AAP,41.12,N/A,-0.060114,0.303965,-0.208165,0.081498,-0.326454,0.017621,-0.207402,0.006608,N/A
3,AAPL,273.81,N/A,0.072629,0.497797,0.358859,0.881057,0.085213,0.687225,-0.018074,0.328194,N/A
4,ABBV,229.89,N/A,0.288693,0.801762,0.251034,0.801762,0.045002,0.596916,0.009618,0.515419,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
449,YUM,154.32,N/A,0.152846,0.662996,0.091449,0.561674,0.032586,0.568282,0.007245,0.506608,N/A
450,ZBH,90.23,N/A,-0.155308,0.187225,-0.01388,0.348018,-0.088402,0.200441,-0.074754,0.066079,N/A
451,ZBRA,245.9,N/A,-0.370536,0.050661,-0.169958,0.112335,-0.202866,0.046256,-0.027102,0.27533,N/A
452,ZION,59.55,N/A,0.095072,0.550661,0.20376,0.755507,0.046389,0.603524,0.11873,0.929515,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [200]:
for row in hqm_dataframe_updated.index:
    percentiles = []
    for time_period in time_periods:
        percentiles.append(hqm_dataframe_updated.loc[row, f'{time_period} Return Percentile'])
    
    hqm_dataframe_updated.loc[row, 'HQM Score'] = mean(percentiles)

hqm_dataframe_updated

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,138.32,N/A,0.027637,0.422907,0.191079,0.740088,0.112076,0.735683,-0.098893,0.03304,0.48293
1,AAL,15.68,N/A,-0.091014,0.264317,0.438532,0.920705,0.313233,0.96696,0.116014,0.920705,0.768172
2,AAP,41.12,N/A,-0.060114,0.303965,-0.208165,0.081498,-0.326454,0.017621,-0.207402,0.006608,0.102423
3,AAPL,273.81,N/A,0.072629,0.497797,0.358859,0.881057,0.085213,0.687225,-0.018074,0.328194,0.598568
4,ABBV,229.89,N/A,0.288693,0.801762,0.251034,0.801762,0.045002,0.596916,0.009618,0.515419,0.678965
...,...,...,...,...,...,...,...,...,...,...,...,...
449,YUM,154.32,N/A,0.152846,0.662996,0.091449,0.561674,0.032586,0.568282,0.007245,0.506608,0.57489
450,ZBH,90.23,N/A,-0.155308,0.187225,-0.01388,0.348018,-0.088402,0.200441,-0.074754,0.066079,0.200441
451,ZBRA,245.9,N/A,-0.370536,0.050661,-0.169958,0.112335,-0.202866,0.046256,-0.027102,0.27533,0.121145
452,ZION,59.55,N/A,0.095072,0.550661,0.20376,0.755507,0.046389,0.603524,0.11873,0.929515,0.709802


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [201]:
hqm_dataframe_updated.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe_updated = hqm_dataframe_updated[:50]
hqm_dataframe_updated.reset_index(inplace=True, drop=True)
hqm_dataframe_updated

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MU,286.68,N/A,2.195274,0.995595,1.348296,0.993392,0.772803,0.997797,0.21228,0.993392,0.995044
1,LRCX,177.33,N/A,1.402846,0.991189,0.935706,0.988987,0.381936,0.984581,0.136731,0.955947,0.980176
2,ALB,148.51,N/A,0.66491,0.955947,1.568488,0.995595,0.820422,1.0,0.142472,0.962555,0.978524
3,NEM,104.73,N/A,1.744497,0.993392,0.761352,0.980176,0.254702,0.931718,0.154304,0.975771,0.970264
4,WDC,179.56,N/A,2.860161,1.0,1.973832,1.0,0.636977,0.995595,0.099369,0.878855,0.968612
5,GM,82.88,N/A,0.576865,0.936123,0.708162,0.971366,0.383178,0.986784,0.127312,0.940529,0.9587
6,HII,355.45,N/A,0.866761,0.975771,0.499156,0.942731,0.280855,0.951542,0.133378,0.951542,0.955396
7,EXPE,286.57,N/A,0.517528,0.920705,0.728199,0.973568,0.315567,0.971366,0.120771,0.936123,0.950441
8,C,121.56,N/A,0.742296,0.964758,0.531754,0.951542,0.195868,0.867841,0.173359,0.977974,0.940529
9,TPR,130.2,N/A,1.011432,0.988987,0.518367,0.947137,0.147946,0.812775,0.191435,0.982379,0.932819


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [165]:
portfolio_input()

In [202]:
position_size = float(portfolio_size) / len(hqm_dataframe_updated.index)
for i in hqm_dataframe_updated.index:
    hqm_dataframe_updated.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe_updated.loc[i, 'Price'])

hqm_dataframe_updated

/var/folders/h9/w84pm_9x5qs91pvj4jj1p5fc0000gp/T/ipykernel_2412/3137620797.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe_updated.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe_updated.loc[i, 'Price'])


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MU,286.68,69,2.195274,0.995595,1.348296,0.993392,0.772803,0.997797,0.21228,0.993392,0.995044
1,LRCX,177.33,112,1.402846,0.991189,0.935706,0.988987,0.381936,0.984581,0.136731,0.955947,0.980176
2,ALB,148.51,134,0.66491,0.955947,1.568488,0.995595,0.820422,1.0,0.142472,0.962555,0.978524
3,NEM,104.73,190,1.744497,0.993392,0.761352,0.980176,0.254702,0.931718,0.154304,0.975771,0.970264
4,WDC,179.56,111,2.860161,1.0,1.973832,1.0,0.636977,0.995595,0.099369,0.878855,0.968612
5,GM,82.88,241,0.576865,0.936123,0.708162,0.971366,0.383178,0.986784,0.127312,0.940529,0.9587
6,HII,355.45,56,0.866761,0.975771,0.499156,0.942731,0.280855,0.951542,0.133378,0.951542,0.955396
7,EXPE,286.57,69,0.517528,0.920705,0.728199,0.973568,0.315567,0.971366,0.120771,0.936123,0.950441
8,C,121.56,164,0.742296,0.964758,0.531754,0.951542,0.195868,0.867841,0.173359,0.977974,0.940529
9,TPR,130.2,153,1.011432,0.988987,0.518367,0.947137,0.147946,0.812775,0.191435,0.982379,0.932819


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [203]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe_updated.to_excel(writer, sheet_name='Momentum Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [204]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [205]:
column_formats = { 
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [206]:
writer.close()